In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm

In [10]:
# Path dataset
train_images_path = 'vehicle-accident/train/images/'
train_labels_path = 'vehicle-accident/train/labels/'

valid_images_path = 'vehicle-accident/valid/images/'
valid_labels_path = 'vehicle-accident/valid/labels/'

output_train_path = 'remove_background/train/'
output_valid_path = 'remove_background/valid/'

os.makedirs(output_train_path, exist_ok=True)
os.makedirs(output_valid_path, exist_ok=True)

In [ ]:
# Function to read bounding box from YOLO label file
def read_bounding_boxes(label_file):
    bounding_boxes = []

    with open(label_file, 'r') as f:
        lines = f.readlines()

        for line in lines:
            parts = line.strip().split()
            class_id = int(parts[0])
            x_center, y_center, width, height = map(float, parts[1:])
            bounding_boxes.append((class_id, x_center, y_center, width, height))
    return bounding_boxes

In [ ]:
# Function to remove background outside the bounding box
def remove_background(image, bounding_boxes):
    h, w, _ = image.shape
    mask = np.zeros_like(image, dtype=np.uint8)

    for _, x_center, y_center, width, height in bounding_boxes:
        x1 = int((x_center - width / 2) * w)
        y1 = int((y_center - height / 2) * h)
        x2 = int((x_center + width / 2) * w)
        y2 = int((y_center + height / 2) * h)
        
        # Fill the mask with the bounding box area
        mask[y1:y2, x1:x2] = image[y1:y2, x1:x2]
        
    return mask

In [ ]:
# Process Dataset
def process_dataset(images_path, labels_path, output_path):
    for img_file in tqdm(os.listdir(images_path)):
        img_path = os.path.join(images_path, img_file)
        label_file = os.path.join(labels_path,
        img_file.replace('.jpg', '.txt'))
        output_file = os.path.join(output_path, img_file)
        
        if not os.path.exists(label_file):
            continue

        # Read images and labels
        image = cv2.imread(img_path)
        bounding_boxes = read_bounding_boxes(label_file)

        # The process to remove the background
        result_image = remove_background(image, bounding_boxes)

        # Save result
        cv2.imwrite(output_file, result_image)

In [11]:
print("Processing train dataset...")
process_dataset(train_images_path, train_labels_path,output_train_path)

print("Processing validation dataset...")
process_dataset(valid_images_path, valid_labels_path,output_valid_path)

print("Proses selesai. Hasil disimpan di remove_background")

Processing train dataset...


100%|██████████| 12620/12620 [03:27<00:00, 60.72it/s]


Processing validation dataset...


100%|██████████| 902/902 [00:14<00:00, 63.29it/s]

Proses selesai. Hasil disimpan di remove_background
